In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 04:06:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2


In [5]:
df_green=spark.read.parquet('../code/data/pq/green/*/*')

In [6]:
df_green.rdd.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 12, 14), lpep_dropoff_datetime=datetime.datetime(2020, 1, 11, 12, 21), store_and_fwd_flag=None, RatecodeID=None, PULocationID=95, DOLocationID=134, passenger_count=None, trip_distance=2.44, fare_amount=8.17, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.47, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), lpep

In [7]:
df_green.rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 12, 14), lpep_dropoff_datetime=datetime.datetime(2020, 1, 11, 12, 21), store_and_fwd_flag=None, RatecodeID=None, PULocationID=95, DOLocationID=134, passenger_count=None, trip_distance=2.44, fare_amount=8.17, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.47, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), lpep

In [8]:
rdd=df_green \
 .select('lpep_pickup_datetime','PULocationID','total_amount') \
 .rdd

In [9]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 12, 14), PULocationID=95, total_amount=8.47),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), PULocationID=117, total_amount=43.91),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [10]:
from datetime import datetime

In [11]:
start=datetime(year=2020,month=1,day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime>=start

In [12]:
def prepare_for_grouping(row):
    hour=row.lpep_pickup_datetime.replace(minute=0,second=0,microsecond=0)
    zone=row.PULocationID

    key=(hour,zone)

    amount=row.total_amount
    count=1

    value=(amount,count)

    return (key,value)

In [13]:
rdd \
 .filter(filter_outliers) \
 .map(prepare_for_grouping) \
 .take(5)

[((datetime.datetime(2020, 1, 23, 13, 0), 74), (44.97, 1)),
 ((datetime.datetime(2020, 1, 11, 12, 0), 95), (8.47, 1)),
 ((datetime.datetime(2020, 1, 30, 12, 0), 117), (43.91, 1)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (8.3, 1)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (12.74, 1))]

In [14]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [15]:
from collections import namedtuple

In [16]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [17]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [18]:
from pyspark.sql import types


In [19]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [20]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [22]:
df_result.write.parquet('tmp/green-revenue',mode='overwrite')


In [23]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [24]:
import pandas as pd


In [25]:
rows = duration_rdd.take(10)

In [26]:
df = pd.DataFrame(rows, columns=columns)

In [27]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [28]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [29]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [30]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [31]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              12.2|
|             65.55|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               8.4|
|              3.85|
|               0.0|
|               6.5|
|               7.5|
|             39.15|
|              15.2|
|              24.8|
|             59.45|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
+------------------+
only showing top 20 rows

